In this part, we'll try to plot the time-course of 1 IC extracted from the EEG of 1 subject on 1 trial.

# First let's extract the time scale data and the time-course data from the .mat file:

In [1]:
import numpy as np
from loader import load

In [2]:
directory = "E:/timot/Documents/1 - Centrale Marseille/0.5 - Semestre S8/Stage/NIC_250819"
file = "FCK_LOCKED_IC_JYOTIKA_250819.mat"

data, n_IC, n_subj, n_trials = load(directory, file, tfr="Comp256Hz")

loading the raw timecourses...
The independent component IC1 of the subject 20 is not in the .mat file.
The independent component IC1 of the subject 21 is not in the .mat file.
The independent component IC2 of the subject 5 is not in the .mat file.
The independent component IC2 of the subject 9 is not in the .mat file.
The independent component IC3 of the subject 1 is not in the .mat file.
The independent component IC3 of the subject 17 is not in the .mat file.
The independent component IC3 of the subject 23 is not in the .mat file.
The independent component IC4 of the subject 12 is not in the .mat file.
Computing and loading the time-frequency wavelet transformation of the 1st trial for all subjects, all IC...
Computing and loading the time-frequency wavelet transformation of the 2nd subject for all IC, 102 trials...
Loaded


# Now we'll implement the HMM method.

### Cumputing functions:

In [1]:
from sklearn.decomposition import PCA
from hmmlearn import hmm
import scipy.signal as signal

In [2]:
# The embedx function copies the `x` array len(lags) times into `xe`
# with lags (i.e. time delays) between lags[0] and lags[-1] (we implement the time-delay array for the HMM).

def embedx(x, lags):
    
    Xe = np.zeros((x.shape[1], x.shape[0],  len(lags)))

    for l in range(len(lags)):
        Xe[:, :, l] = np.roll(x, lags[l], axis=0).swapaxes(0, 1)

    # Remove edges
    valid = np.ones((x.shape[0], 1), dtype=np.int8)
    valid[:np.abs(np.min(lags)), :] = 0
    valid[-np.abs(np.max(lags)):, :] = 0

    Xe = Xe[:, valid[:, 0] == 1, :]

    return Xe, valid


# The hmm_tde function finds parameters for the HMM,
# then uses them to determine the probability of presence of each found state over time.

def hmm_tde(data: np.array, lags, n_states=3, n_iter=100, n_components=8, 
            covariance_type='full', model_type='GMMHMM', tol=0.01, n_mix=1, **kwargs):
    
    # Embed time serie
    xe, valid = embedx(data, lags)

    pca = PCA(n_components=n_components)
    y = pca.fit_transform(xe[0, :, 3:])
    print(y.shape)
    
    if model_type=='GMMHMM':
        model = hmm.GMMHMM(n_components=n_states, n_iter=n_iter,
                            covariance_type=covariance_type, tol=tol, n_mix=n_mix, **kwargs)
        
    elif model_type=='GaussianHMM':
        model = hmm.GaussianHMM(n_components=n_states, n_iter=n_iter,
                            covariance_type=covariance_type, tol=tol, **kwargs)
        
    elif model_type=='MultinomialHMM':
        model = hmm.MultinomialHMM(n_components=n_states, n_iter=n_iter, tol=tol, **kwargs)
    
    else: 
        return "Non-exixting model_type. Please choose 'GMMHMM' or 'GaussianHMM' or 'MultinomialHMM'. default='GMMHMM'"
        
    model.fit(y)
    gamma = model.predict_proba(y)
    
    return gamma, model, xe, y

NameError: name 'np' is not defined

### Plotting functions:

In [ ]:
import matplotlib.pyplot as plt
# from plotter import *

In [ ]:
def show_bigstates(
    gamma, n_states, # the data we need for the plot
    
    subj, IC, trial, # which IC of which subject is of interest here, how many trials
    
    n_components, n_lags, covariance_type, model_type, n_mix, # infos we put in the .png name if we want to save it
    
    save=False # do we really want to save the figure?
):

    fig = plt.figure(figsize=(6, 6))
    plt.title('State Power Spectrum')
    max_power = np.zeros(n_states)
    max_freq = np.zeros(n_states)
    for i in range(n_states):

        # Compute PSD separately for each lag
        tot = []
        for seg in xe[0, gamma[:, i]> .6, :].T:
            freqs, psd = signal.welch(x=seg, fs=256, nfft=1000)
            tot.append(psd)
        psd = np.mean(np.asarray(tot), 0)
        
        max_freq[i] = freqs[np.argmax(psd[250:])]
        max_power[i] = np.amax(psd[250:])

        plt.plot(freqs, psd)
        print(psd.shape)
    #     plt.fill_between(x=freqs, y1=psd, alpha=0.3)

    plt.xlim(0, 50)
    plt.ylabel('PSD')
    plt.xlabel('Frequencies (Hz)')
    plt.legend([f'state {i+1}' for i in range(n_states)], loc='upper right')
    plt.tight_layout()   

    if save==True:
        plt.savefig(f'tde-hmm2/su{subj}IC{IC}tr{trial}_co{n_components}lg{n_lags}st{n_states}'
                    +covariance_type+f'{n_mix}'+model_type+'__noalpha_big-states-info.png', dpi=600)
    
#     plt.close(fig)
    
    return max_freq, max_power
    

In [ ]:
def plot_hmm_over_bigtfr(   
    bigtime, bigtfr, gamma, lags, n_states, max_power, # the data we need for the plot
    
    subj, IC, trial, # which IC of which subject is of interest here, how many trials
    
    n_components, n_lags, covariance_type, model_type, n_mix, # infos we put in the .png name if we want to save it
    
    save=False # do we really want to save the figure?
):
    
    fig = plt.figure(figsize=(16*3, 5))

    # HMM states probability plot
    plt.subplot(211)
    plt.title('HMM States probability')
    time = bigtime[np.abs(np.min(lags)):-np.abs(np.max(lags))]

    burst = np.argmax(max_power) # this is the burst state index
    
    labels = ['']
    states = np.where([i not in [burst] for i in range(n_states)])[0] # the other indexes
    for i in states:
        plt.fill_between(x=time[:1793*3-48], y1=gamma[:1793*3-48, i], alpha=0.05)
        plt.xlim(0, 7*3)
        labels.append(f'state {i+1}')
    plt.plot(time[:1793*3-48], gamma[:1793*3-48, burst]>0.6, 'red')
    labels[0]=(f'burst state (state {burst+1})')
    plt.fill_between(x=time[:1793*3-48], y1=gamma[:1793*3-48, burst], alpha=0.2, color='red')
    plt.ylabel('State probability')
    plt.legend(labels, loc='upper left')

    # Time-frequency plot
    plt.subplot(212)

    plt.title('Wavelet transform')
    plt.imshow(bigtfr[0, :, :1793*3], vmax=4,
               aspect='auto', origin='lower', extent=[0, 7*3, 2, 50], cmap='RdBu_r')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequencies (Hz)')
    plt.tight_layout()
    
    if save == True:
        plt.savefig(f'tde-hmm2/su{subj}IC{IC}tr{trial}_co{n_components}lg{n_lags}st{n_states}'
                    +covariance_type+f'{n_mix}'+model_type+'_noalpha_big-tde-hmm.png', dpi=600)
        
#     plt.close(fig)

### Using the HMM method on a big timecourse (several variations)

#### Variation1 : data of 1IC, keeping a highly reducting PCA before TDE-HMM
subj2, IC1, 10 first trials concatenated.

|_ 19delays x (7s x 256Hz x 10trials)   -----PCA-----> |_ 8components x (7s x 256Hz x 10trials) -----> TDE-HMM 
                                                                                                (
                                                                                                1 Gaussian/state,
                                                                                                3 states
                                                                                                )

In [ ]:
# The data we want to analyse with the HMM method
subj=2
IC=1
trial=10    # Number of trials in the timecourse

big_timecourse = np.concatenate([data[f'raw_timecourse_256Hz subject{subj}, IC{IC}'][i] for i in range(trial)])
print(big_timecourse.shape)

x = big_timecourse.reshape(-1, 1)

In [ ]:
# The parameters we change to hope for some results
lags = np.arange(-29, 29)
n_lags = lags.shape[0]
n_iter=100
n_states=3    # for the Hidden Markov Model
n_components=40     # For the principal component analysis
covariance_type='diag'
model_type='GMMHMM'
tol=0.01
n_mix=1
save=True

In [ ]:
# Creating a big time axis
time = data['time_axis']
bigtime = np.concatenate([time+4+(7*i) for i in range(100)])
bigtime[np.abs(np.min(lags)):-np.abs(np.max(lags))].shape

In [ ]:
# Finding the model
gamma, model, xe, y = hmm_tde(x, lags, n_iter=n_iter, n_states=n_states, n_components=n_components, 
                           covariance_type=covariance_type, model_type=model_type, tol=tol, n_mix=n_mix)

In [3]:
max_freq, max_power = show_bigstates(
    gamma, n_states, # the data we need for the plot
    
    subj, IC, trial, # which IC of which subject is of interest here, how many trials
    
    n_components, n_lags, covariance_type, model_type, n_mix, # infos we put in the .png name if we want to save it
    
    save # do we really want to save the figure?
)

NameError: name 'show_bigstates' is not defined

In [4]:
# Concatenating the tfr to compare them with the big state probability timecourse
print(data['tfr_256Hz subject2, IC1, trial1'].shape)

bigtfr = np.concatenate([data[f'tfr_256Hz subject2, IC1, trial{i+1}'] for i in range(100)], axis=2)
print(bigtfr.shape)

NameError: name 'data' is not defined

In [5]:
plot_hmm_over_bigtfr(   
    bigtime, bigtfr, gamma, lags, n_states, max_power, # the data we need for the plot
    
    subj, IC, trial, # which IC of which subject is of interest here, how many trials
    
    n_components, n_lags, covariance_type, model_type, n_mix, # infos we put in the .png name if we want to save it
    
    save # do we really want to save the figure?
)

NameError: name 'plot_hmm_over_bigtfr' is not defined

In [6]:
plt.figure()
plt.imshow(xe[0,:1000,:].T, aspect='auto')

NameError: name 'plt' is not defined

In [7]:
plt.figure(figsize=(30,12))
plt.subplot(311)
plt.imshow(y[:1000,:].T, aspect='auto')
plt.subplot(312)
plt.plot(bigtime[:1000], y[:1000, 0])
plt.subplot(313)
plt.plot(bigtime[:1000], y[:1000, 7])
'''
filtres passe haut
lire papier vs these
normaliser les puissances dans y
rôle pca
'''

NameError: name 'plt' is not defined